In [1]:
import json
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

## First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

### Read receipts

In [2]:
# Opening receipts JSON file

data = []
with open('receipts.json') as f:
    for line in f:
        data.append(json.loads(line))


In [3]:
# Convert to dataframe

receipts = pd.DataFrame(data)

In [4]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [5]:
receipts.shape

(1119, 15)

In [6]:
# Expand rewardsReceiptItemList to seperate columns

receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)

In [7]:
receipts.shape

(7381, 16)

In [8]:
# normalize and rename dictionary columns 

receipt_id=pd.json_normalize(receipts['_id'])
receipt_id.rename(columns = {'$oid':'id'}, inplace = True)

receipt_createDate=pd.json_normalize(receipts['createDate'])
receipt_createDate.rename(columns = {'$date':'createDate'}, inplace = True)

receipt_datescanned=pd.json_normalize(receipts['dateScanned'])
receipt_datescanned.rename(columns = {'$date':'dateScanned'}, inplace = True)

receipt_finished=pd.json_normalize(receipts['finishedDate'])
receipt_finished.rename(columns = {'$date':'finishedDate'}, inplace = True)

receipt_modify=pd.json_normalize(receipts['modifyDate'])
receipt_modify.rename(columns = {'$date':'modifyDate'}, inplace = True)

receipt_point=pd.json_normalize(receipts['pointsAwardedDate'])
receipt_point.rename(columns = {'$date':'pointsAwardedDate'}, inplace = True)

receipt_purchase=pd.json_normalize(receipts['purchaseDate'])
receipt_purchase.rename(columns = {'$date':'purchaseDate'}, inplace = True)

receipt_rewardsReceiptItemList=pd.json_normalize(receipts['rewardsReceiptItemList'])


In [9]:
receipt_rewardsReceiptItemList.columns

Index(['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')

In [10]:
receipts_new = pd.concat([receipt_id, receipts.iloc[:, 2:4], receipt_createDate, receipt_datescanned,
                          receipt_finished, receipt_modify, receipt_point, receipts['pointsEarned'], receipt_purchase, 
                         receipts['purchasedItemCount'], receipt_rewardsReceiptItemList, receipts.iloc[:, 13:16]], axis=1)


In [11]:
receipts_new.shape

(7381, 48)

In [12]:
receipts_new.columns

Index(['id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount', 'barcode',
       'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       '

In [13]:
receipts_new.head()

,id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,...,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
3,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5,1.609632e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
4,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6


In [14]:
# Convert timestamp to UTC time

receipts_new['createDate']=receipts_new['createDate'].apply(lambda x: datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else x)
receipts_new['dateScanned']=receipts_new['dateScanned'].apply(lambda x: datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else x)
receipts_new['finishedDate']=receipts_new['finishedDate'].apply(lambda x: datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else x)
receipts_new['modifyDate']=receipts_new['modifyDate'].apply(lambda x: datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else x)
receipts_new['pointsAwardedDate']=receipts_new['pointsAwardedDate'].apply(lambda x: datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else x)
receipts_new['purchaseDate']=receipts_new['purchaseDate'].apply(lambda x: datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else x)




In [15]:
receipts_new.head()

,id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,...,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
3,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaN,2021-01-03 15:25:42,NaN,5,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
4,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6


### Read Users

In [16]:
# Opening users JSON file

user = []
with open('users.json') as f:
    for line in f:
        user.append(json.loads(line))
        
# Convert to dataframe

users = pd.DataFrame(user)

In [17]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [18]:
# normalize and renamedictionary columns 

user_id=pd.json_normalize(users['_id'])
user_id.rename(columns = {'$oid':'id'}, inplace = True)

user_createdDate=pd.json_normalize(users['createdDate'])
user_createdDate.rename(columns = {'$date':'createdDate'}, inplace = True)

user_lastLogin=pd.json_normalize(users['lastLogin'])
user_lastLogin.rename(columns = {'$date':'lastLogin'}, inplace = True)

In [19]:
# concat the dataframes

users_new = pd.concat([user_id, users.iloc[:, 1], user_createdDate, user_lastLogin,users.iloc[:, 4:7]], axis=1)

In [20]:
users_new.head()

,id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI


In [21]:
users_new['createdDate']=users_new['createdDate'].apply(lambda x: datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else x)
users_new['lastLogin']=users_new['lastLogin'].apply(lambda x: datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else x)



In [22]:
users_new.head()

,id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


### Read brands

In [23]:
# Opening users JSON file

brand = []
with open('brands.json') as f:
    for line in f:
        brand.append(json.loads(line))
        
# Convert to dataframe

brands = pd.DataFrame(brand)

In [24]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,False,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,False,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,False,TEST BRANDCODE @1612366146827


In [25]:
brands.shape

(1167, 8)

In [26]:
# normalize and renamedictionary columns 

brand_id=pd.json_normalize(brands['_id'])
brand_id.rename(columns = {'$oid':'id'}, inplace = True)

brand_cpg=pd.json_normalize(brands['cpg'])
brand_cpg.rename(columns = {'$ref':'cpg_ref','$id.$oid': 'cpg_id'}, inplace = True)

In [27]:
# concat the dataframes

brands_new = pd.concat([brand_id, brands.iloc[:, 1:3], brand_cpg, brands.iloc[:, 5:8]], axis=1)

In [28]:
brands_new.head()

,id,barcode,category,cpg_ref,cpg_id,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,Cogs,601ac114be37ce2ead437550,test brand @1612366101024,False,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,Cogs,5332f5fbe4b03c9a25efd0ba,Starbucks,False,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,Cogs,601ac142be37ce2ead437559,test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,Cogs,601ac142be37ce2ead437559,test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,Cogs,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,False,TEST BRANDCODE @1612366146827


I used Lucid Chart to draw the relational diagram based on the structured data above. 
https://lucid.app/lucidchart/f07410c1-bbc9-4bf9-a2dc-71d74df18013/edit?viewport_loc=-23%2C-10%2C1946%2C892%2C0_0&invitationId=inv_8b2381c2-3a00-4a6c-9822-99364abf5843#


## Second: Write a query that directly answers a predetermined question from a business stakeholder

In [29]:
# Convert dataframe to SQL table

engine = create_engine('sqlite://', echo=False)

receipts_new.to_sql('receipts',con=engine)
brands_new.to_sql('brands',con=engine)
users_new.to_sql('users',con=engine)


In [30]:
# Q1. What are the top 5 brands by receipts scanned for most recent month?


# To answer Q1, we need to know when is the most recent month first

print(engine.execute("SELECT distinct substring(dateScanned, 1, 7) as time from receipts order by time desc").fetchall())  


[('2021-03',), ('2021-02',), ('2021-01',), ('2020-11',), ('2020-10',)]


In [31]:
print(engine.execute('''SELECT b.name, count(r.id) as receipts_num 
                     from receipts r 
                     join brands b 
                     on r.brandCode = b.brandCode
                     where substring(r.dateScanned, 1, 7) = "2021-03"
                     group by b.name 
                     order by receipts_num desc 
                     limit 5 ''').fetchall())  



[]


In [32]:
# It seems there is no brandcode in Mar 2021, let's see if this is a mapping problem

print(engine.execute("SELECT distinct substring(dateScanned, 1, 7) as time, count(distinct brandCode) from receipts group by time").fetchall())  



[('2020-10', 0), ('2020-11', 0), ('2021-01', 226), ('2021-02', 3), ('2021-03', 0)]


So we can see that there is no brandcode in March. Let's also check other months.

In [33]:
# Check the brand name for Feb 2021

print(engine.execute('''SELECT b.name, count(r.id) as receipts_num 
                     from receipts r 
                     join brands b 
                     on r.brandCode = b.brandCode
                     where substring(r.dateScanned, 1, 7) = "2021-02"
                     group by b.name 
                     order by receipts_num desc 
                     limit 5 ''').fetchall())  

[('Viva', 1)]


In [34]:
# Get all the unique brandcode in Feb 2021

print(engine.execute("SELECT distinct brandCode from receipts where substring(dateScanned, 1, 7) = '2021-02'").fetchall())  


[(None,), ('BRAND',), ('MISSION',), ('VIVA',)]


As we can see that there should have three distinct brandcodes in Feb 2021, but we only get 1 brandname from brands table, which means there are more unique brandcodes in receipts table than in brands table. So this is one of the data quality issuse we need to pay attention to in the third step later.

In [35]:
# Q2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking 
# for the previous month?

print(engine.execute("SELECT distinct brandCode as brandcode, count(id) from receipts where substring(dateScanned, 1, 7) = '2021-02' group by brandcode").fetchall())  


[(None, 509), ('BRAND', 3), ('MISSION', 2), ('VIVA', 1)]


We have answered this question in Q1. There is no brandcode Mar 2021 and only three brandcodes in Feb 2021. The ranking for the three brandcodes in Feb 2021 are BRAND, MISSION and VIVA.

In [36]:
# Q3. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, 
# which is greater?

print(engine.execute('''select a.rewardsReceiptStatus, round(avg(a.spent),2)
                        from (
                        SELECT distinct id as id, max(totalSpent) as spent, rewardsReceiptStatus
                        from receipts 
                        group by id) a
                        group by rewardsReceiptStatus''').fetchall())               

[('FINISHED', 80.85), ('FLAGGED', 180.45), ('PENDING', 28.03), ('REJECTED', 23.33), ('SUBMITTED', None)]


I assume the 'Accepted' in the question means 'FINISHED' here. So we can see the average spend of FINISHED is greater than that of REJECTED.

In [37]:
# Q4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ 
# or ‘Rejected’, which is greater?

print(engine.execute('''select a.rewardsReceiptStatus, round(sum(purchase),2)
                        from (
                        SELECT distinct id as id, max(purchasedItemCount) as purchase, rewardsReceiptStatus
                        from receipts 
                        group by id) a
                        group by rewardsReceiptStatus''').fetchall())               

[('FINISHED', 8184.0), ('FLAGGED', 1014.0), ('PENDING', None), ('REJECTED', 173.0), ('SUBMITTED', None)]


I assume the 'Accepted' in the question means 'FINISHED' here. So we can see the total number of items purchased of FINISHED is greater than that of REJECTED.

In [38]:
# Q5. Which brand has the most spend among users who were created within the past 6 months?

# Let's first see the year and month of createdDate column in Users table.

print(engine.execute("SELECT distinct substring(createdDate, 1, 7) as createdtime from users order by createdtime").fetchall())  


[('2014-12',), ('2015-04',), ('2017-07',), ('2017-09',), ('2017-12',), ('2020-01',), ('2020-07',), ('2020-11',), ('2020-12',), ('2021-01',), ('2021-02',)]


Since the most recent month in receipts table is Mar 2021, we assume that it is now March 2021, so the last six months would be from October 2020.

In [39]:
print(engine.execute('''
                        SELECT r.brandCode, round(sum(r.finalPrice),2)
                        from receipts r
                        where r.userId in (
                           SELECT id 
                           from users u 
                           where substring(createdDate, 1, 7) between '2020-10' and '2021-03')
                        group by brandCode
                        order by 2 desc
                        limit 5''').fetchall()) 

[(None, 21092.27), ('BEN AND JERRYS', 1217.4), ("HEMPLER'S", 800.55), ('CRACKER BARREL', 703.5), ('HY-VEE', 656.62)]


We can see since a lot of items do not have brandcode, the brand that has the most spend among users who were created within the past 6 months is BEN AND JERRYS.

In [40]:
# Q6. Which brand has the most transactions among users who were created within the past 6 months?


print(engine.execute('''
                        SELECT r.brandCode, count(r.brandCode)
                        from receipts r
                        where r.userId in (
                           SELECT id 
                           from users 
                           where substring(createdDate, 1, 7) between '2020-10' and '2021-03')
                        group by brandCode
                        order by 2 desc
                        limit 5''').fetchall()) 

[('HY-VEE', 291), ('BEN AND JERRYS', 100), ('PEPSI', 74), ('KLEENEX', 70), ('KNORR', 60)]


The brand that has the most transactions among users who were created within the past 6 months is HY-VEE.

## Third: Evaluate Data Quality Issues in the Data Provided

### 1. Check duplicate records

In [41]:
# calculate duplicates for users

len(users_new)-len(users_new.drop_duplicates())

283

There are 495 records in users file, and more than half (283) records are dulicate rows. 

In [42]:
# calculate duplicates for brands

len(brands_new)-len(brands_new.drop_duplicates())

0

In [43]:
# calculate duplicates for receipts

len(receipts_new)-len(receipts_new.drop_duplicates())

0

There is no duplicate records in receipts and brands file.

Duplicated records will increase the size of the file and cause problem when joining two tables . So it's better to remove duplicated records.

### 2. Missing keys

When answering Q1, we found that some brandcodes are not in brand file. This will cause problems when we join tables. So here we are going to check the missing brandcodes in brands table and missing userID in users table since these two variables are the foreign key in the receipt table to join the other two tables.

In [44]:
# Find brandcodes in receipts but not in brands

for i in receipts_new['brandCode'].unique():
    if i not in list(brands['brandCode'].unique()):
        print([i])

['MISSION']
['BRAND']
['KRAFT EASY CHEESE']
['WINGSTOP']
['GERM-X']
['BEN AND JERRYS']
['BORDEN']
['KLARBRUNN']
['HY-VEE']
['LIGHT & FIT GREEK']
["CONNIE'S PIZZA"]
["VAN DE KAMP'S"]
['HATCH FARMS']
["KELLOGG'S"]
['TEMPTATIONS']
["NATURE'S PATH ORGANIC"]
['DOLE']
['EL MONTEREY']
['BIGELOW']
['HY-VEE SELECT']
['KIKKOMAN']
['SPECIAL K']
['HILLSHIRE FARM']
['JUST BARE']
["LAURA'S LEAN BEEF"]
['CAL-ORGANIC FARMS']
["BUSH'S BEST"]
['FOLGERS']
['KASHI']
['LIPTON']
['GREEN GIANT']
['HARVEST SNAPS']
["THAT'S SMART!"]
['ADVIL']
['CHICKEN OF THE SEA']
['STARKIST']
['TIC TAC']
['SO DELICIOUS']
['WONDERFUL']
['LIGHT & FIT']
['HANOVER']
['HIDDEN VALLEY']
['DANNON']
['FAGE']
['ORAL-B GLIDE']
["CAMPBELL'S"]
["FRENCH'S"]
['CRISPIX']
['KING ARTHUR FLOUR']
['KITCHEN BASICS']
['MCCORMICK']
['OLD EL PASO']
['ZESTA']
['AZTECA']
['BUNNY']
['HONEY BUNCHES OF OATS']
['SIMPLE TRUTH ORGANIC']
['BOTA BOX']
['DARE']
['LINDT']
['ORGANIC ROOT STIMULATOR']
["MERKT'S"]
['MORTON']
['FRONTERA']
['KARO']
['CHEESE']
["FLO

In [45]:
# Similarly, find userId in receipts but not in users

for i in receipts_new['userId'].unique():
    if i not in list(users_new['id'].unique()):
        print([i])

['5f9c74f7c88c1415cbddb839']
['5ff1e1e9b6a9d73a3a9f10f6']
['5ff1e1dfcfcf6c399c274ab3']
['5f9c74e3f1937815bd2c1d73']
['5ff1e196cfcf6c399c274a38']
['5ff36d78135e7011bcb86488']
['5c3388caea88e15513a95069']
['5ff37124135e7011bcb86bc3']
['5ff473f3c1e2d0121a9b2707']
['5ff4ce68c1e2d0121a9b3022']
['5fa5ad376a26f611e71ab5ef']
['5ff4ce65c3d63511e2a4853b']
['5ff4ce3cc1e2d0121a9b2fba']
['5ff5d19b8f142f11dd188696']
['5ff73be5eb7c7d31ca8a45a7']
['5ff7741e04929111f6e90902']
['5ff79459b3348b11c933736d']
['5ff75004b3348b11c9336bd6']
['5ff73be9eb7c7d31ca8a45bc']
['5ff7946004929111f6e90ceb']
['5ff726a38f142f11dd1895dc']
['5ff8da4e04929111f6e91462']
['5ff8da7c04929111f6e914de']
['5ff8da65b3348b11c9337b29']
['5ff8da62b3348b11c9337b21']
['5fa5b0b720dc5111dd86dcc1']
['5ff8c88604929111f6e913b7']
['5ff873d304929111f6e91096']
['5ffc98b604929111f6e923f5']
['5ffc9d8cb3348b11c9338927']
['5ffc9da0b3348b11c9338951']
['5ffc8cc104929111f6e922a3']
['5ffcb4b9b3348b11c9338ae7']
['5ffca2f3b3348b11c9338a10']
['5ffc92a10492

Therefore, I recommend updating these brandcode and userId in brands and users file.

### 3. Missing values within the table

In [46]:
receipts_new.isnull().sum()

id                                       0
bonusPointsEarned                     1401
bonusPointsEarnedReason               1401
createDate                               0
dateScanned                              0
finishedDate                          1411
modifyDate                               0
pointsAwardedDate                     1301
pointsEarned                          1128
purchaseDate                           458
purchasedItemCount                     484
barcode                               4291
description                            821
finalPrice                             614
itemPrice                              614
needsFetchReview                      6568
partnerItemId                          440
preventTargetGapPoints                7023
quantityPurchased                      614
userFlaggedBarcode                    7044
userFlaggedNewItem                    7058
userFlaggedPrice                      7082
userFlaggedQuantity                   7082
needsFetchR

We can see that a lot of variables, especially those separated from "rewardsReceiptItemList", have many missing values. When answering questions in the second step, we analyzed spent and transactions by brands. However, here we see that brandcode has 4781 missing values, which makes our analysis inaccurate. 

In [47]:
users_new.isnull().sum()

id               0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [48]:
brands_new.isnull().sum()

id             0
barcode        0
category     155
cpg_ref        0
cpg_id         0
name           0
topBrand     612
brandCode    234
dtype: int64

Users and Brands table also have some missing values. Brandcode is the variable used to join brands table and receipts table, while we can see it has 234 missing values. And topBrand has 612 missing values. I recommend filling these missing values as much as possible.

### 4. Data Inaccuracy

In [49]:
print(engine.execute('''
                        select a.id, round(a.num - b.num2,2)  as diff from (
                        SELECT id, sum(finalPrice) as num
                        from receipts 
                        group by id) a
                        join (SELECT id, totalSpent as num2
                        from receipts) b
                        on a.id=b.id
                        group by 1
                        having diff>0

                        ''').fetchall())  

[('5ff1e1d20a7214ada1000561', 2.56), ('5ff726810a720f05230005ea', 2.56), ('5ff874050a7214ada100065d', 2.56), ('5ffc9d9c0a7214adca00004b', 14.0), ('5ffcb4ad0a720f0515000009', 2.56), ('5ffcb4b80a7214ad4e00000d', 12.0), ('6008888f0a720f05fa0000b5', 2.56), ('6011f39c0a720f05350000b4', 2.56), ('60132b740a7214ad50000012', 2.56), ('601455820a720f05f80000fb', 16.0), ('60145a510a7214ad50000086', 15.0), ('60189c900a7214ad28000038', 17.0), ('602558a90a720f05a8000240', 2.56)]


When answering Q5, "Which brand has the most spend among users who were created within the past 6 months?", I used the finalprice column to calculate the total spend for each brandcode, assuming the sum of the finalprice under each receipt equals to the totalprice column in each receipt.

However, as we queried the receipt which has difference between the totalprice and the sum of finalprice, we found that the receipts listed above have difference between the two. Therefore I wonder if there is any problem in either the totalprice column or the finalprice column considering the differences are all positive numbers.

## Fourth: Communicate with Stakeholders

Hello everyone, 

Hope you are doing well! I have worked on receipts, brands and users json file these days and want to report the issues I found during the analysis.

1. Duplicate records. When I briefly browsed the user file, I found a lot of duplicate rows, then I counted the number of duplicate rows and found that there were 283 duplicates out of a total of 495 records. I would like to know why more than half of the records are duplicates, or more specifically, how the user data is recorded and stored.
    
2. Missing keys and missing values. When I extract data from two files at the same time, I need to connect the two files using a variable that exists in both files, such as brandCode in receipts and brands file. However, I found that some brandCode in the receipt file is not in the brand file. Similarly, some of the userId's are in the receipt file but not in the users file. The brand and user files are supposed to record the complete brand and user information, but due to the data validity, I could not get the complete data, which would lead to inaccurate query results. Besides, all three json files have more or less missing values, which would also lead to inaccurate query results. Therefore, I strongly recommend to update the missing data in these three files.

3. Data inaccuracy. When I aggregated the spend for each brand, I used the finalPrice column in the receipt file, but since I wasn't sure if the finalPrice was what the customer really spent, I also added up the finalPrice of all the items under each receipt and compared them to the totalPrice column to see if there were any differences. As a result, I did find a difference between the sum of the finalPrices and the totalPrice for some receipts. So I want to know if the finalPrice is what the customer is really spending, or more directly, how to break down the totalprice to each brand.

In addition to the issues raised above, I have questions such as :

4. How are these three json files created?

5. The rewardsReceiptItemList column in the receipt file has nested jsons. Since this column contains a lot of information, is it possible to create a separate table for it and assign the receipt ID to it, which would make data cleaning and data analysis easier?

Could we set up a meeting to discuss these issues in the next couple of weeks? Any suggestions or thoughts are highly appreciated. Let me know your availability. Thanks!

Best,
Minqi

